<a href="https://colab.research.google.com/github/sean-halpin/dialoGPT_Virtual_Character/blob/main/dialog_with_persona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install transformers
!pip install flair
!pip install word2number

     |████████████████████████████████| 4.0 MB 4.1 MB/s 
     |████████████████████████████████| 6.5 MB 50.7 MB/s 
     |████████████████████████████████| 895 kB 57.1 MB/s 
     |████████████████████████████████| 596 kB 62.1 MB/s 
     |████████████████████████████████| 77 kB 5.5 MB/s 
     |████████████████████████████████| 400 kB 3.8 MB/s 
     |████████████████████████████████| 46 kB 4.2 MB/s 
     |████████████████████████████████| 1.2 MB 50.2 MB/s 
     |████████████████████████████████| 981 kB 48.3 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 19.7 MB 1.3 MB/s 
     |████████████████████████████████| 1.6 MB 52.9 MB/s 
     |████████████████████████████████| 788 kB 60.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 48 kB 4.8 MB/s 
     |████████████████████████████████| 199 kB 46.4 MB/s 
     |████████

# Chatbot Persona 

In [2]:
persona = {
    "age inquiry": "39"
}

# Load pre-trained dialog req/res model

In [3]:
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("shalpin87/dialoGPT-homer-simpson")
model = AutoModelWithLMHead.from_pretrained("shalpin87/dialoGPT-homer-simpson")

Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:911: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/891 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/487M [00:00<?, ?B/s]

# Load a pre-trained zero-shot classifier  

In [4]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [5]:
candidate_labels = ['age inquiry', 'job inquiry', 'name inquiry', 'statement', 'personal detail']

In [6]:
def classify(sequence_to_classify):
  class_result = classifier(sequence_to_classify, candidate_labels)
  return {
      'label':class_result['labels'][0], 
      'score':class_result['scores'][0]
      }

# Load pre-trained NER (Named Entity Recognition) Model

In [7]:
from flair.data import Sentence
from flair.models import SequenceTagger
tagger = SequenceTagger.load("flair/ner-english-ontonotes-fast")

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

2022-04-07 21:26:07,786 loading file /root/.flair/models/ner-english-ontonotes-fast/0d55dd3b912da9cf26e003035a0c269a0e9ab222f0be1e48a3bbba3a58c0fed0.c9907cd5fde3ce84b71a4172e7ca03841cd81ab71d13eb68aa08b259f57c00b6
2022-04-07 21:26:15,885 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [8]:
from word2number import w2n

In [9]:
def tokens_to_list(tokens):
  tkns = []
  for t in tokens:
    tkns.append(t.text)
  return tkns

In [10]:

def replace_personal_info(response_str, personal_info_type, persona):
  # make example sentence
  sentence = Sentence(response_str)
  list_tkns = tokens_to_list(sentence)
  # predict NER tags
  tagger.predict(sentence)
  # iterate over entities and print
  for entity in sentence.get_spans('ner'):
    if personal_info_type == "age inquiry":
      # print(entity.labels)
      # print(entity.id_text)
      # print(entity.to_dict())
      # print(entity.tokens)
      for t in entity.tokens:
        for l in t.get_labels():
          print(l)
          if "DATE" in l.value or "CARDINAL" in l.value:
            try:
              if t.text.isnumeric() or w2n.word_to_num(t.text).isnumeric():
                # print(t.text)
                # print(t.idx)
                list_tkns[t.idx - 1] = persona['age inquiry']
                return list_tkns
            except Exception as e:
              print("Error: {} - {}".format(e, t))
  return list_tkns

# Test Replacement of Personal Token

In [ ]:
print(" ".join(replace_personal_info("I am 88 years old", "age inquiry", persona)))
print(" ".join(replace_personal_info("I'm 19, what should I know?", "age inquiry", persona)))
print(" ".join(replace_personal_info("20 , you ??", "age inquiry", persona)))

# Chat

In [14]:
questions = [
    "What is your name?",
    "Who are you?",
    "Where do you work?",
    "Who really killed Mr Burns?",
    "Have you ever stolen from the Kwik-E-Mart?",
    "Did you kill Frank Grimes?",
    "Who was the worst member of the Be Sharps?",
    "Hey where did Barney go?",
    "What is your favorite bar to have a beer?",
    "What is the best beer in Springfield?",
    "Is Bart working for the Mob?",
    "I think there was an incident in sector 7 G",
    "Is Ned Flanders house okay?",
    "Oh my god it's Sideshow Bob",
    "What is a Flaming Moe?",
    "What is happening to Apu?",
    "Who quit the band?",
    "What age are you?",
    "How old are you?"
]

botname = "HomerBot"
# Let's chat
for step in range(len(questions)):
    print("***************************************")
    # model_input = input(">> User:")
    model_input = questions[step]
    print("Q. {}".format(model_input))
    classification = classify(model_input)
    new_user_input_ids = tokenizer.encode(model_input + tokenizer.eos_token, return_tensors='pt')

    bot_input_ids = new_user_input_ids

    num_return_seqs=1

    chat_history_ids = model.generate(
        bot_input_ids, 
        max_length=200,
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=3,       
        do_sample=True, 
        top_k=200, 
        top_p=0.55,
        temperature = 0.85,
        num_return_sequences=num_return_seqs
    )
    
    botname = "HomerBot"
    for i in range(0,num_return_seqs):
      bot_output = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][i])
      if classification['label'] in persona:
        try:
          print("\t{}\n\t{}\n\t{}".format(bot_output.replace("<|endoftext|>",""), classification['label'], persona))
          bot_output = " ".join(replace_personal_info(bot_output, classification['label'], persona))
        except Exception as e:
          print(e)
      print("{}: {}".format(botname, bot_output.replace("<|endoftext|>",""), skip_special_tokens=True))

    chat_history_ids = []

***************************************
Q. What is your name?
HomerBot: I don't know.
***************************************
Q. Who are you?
HomerBot: Homer Simpson.
***************************************
Q. Where do you work?
HomerBot: I work at a nuclear power plant.
***************************************
Q. Who really killed Mr Burns?
HomerBot: The guy with the spiked can.
***************************************
Q. Have you ever stolen from the Kwik-E-Mart?
HomerBot: No.
***************************************
Q. Did you kill Frank Grimes?
HomerBot: I'm afraid not.
***************************************
Q. Who was the worst member of the Be Sharps?
HomerBot: Homer Simpson, sir.
***************************************
Q. Hey where did Barney go?
HomerBot: I heard he lost his marbles.
***************************************
Q. What is your favorite bar to have a beer?
HomerBot: The Flanders' O'Toole.
***************************************
Q. What is the best beer in Springfield?
